# Logistic Regression

In this notebook, we apply Logistic Regression to our data and we try to predict 'churn'.

In [1]:
# All imports will be here:
import pandas as pd
import numpy as np
from utils import import_and_transform
from utils import evaluate_model
from utils import aggregate
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    precision_recall_curve,
)
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

Based on the exploratory data analysis **EDA**, we will now modify our database accordingly. The EDA showed issues and necessary changed that require database modifications.

We restructurate our database:

In [2]:
# Performing data import and basic preprocessing:


def import_and_transform(data):

    if isinstance(data, str):
        df = pd.read_parquet(data)
    else:
        df = data

    # Remove invalid userIds and convert valid ones to integers
    df = df[df["userId"] != ""]
    df["userId"] = df["userId"].astype(int)

    # Ecnode the catgeorical variables
    df["gender"] = df["gender"].map({"F": 0, "M": 1})
    df["level"] = df["level"].map({"free": 0, "paid": 1})

    # Convert the timestamps
    df["ts"] = pd.to_datetime(df["ts"], unit="ms")
    df["registration"] = pd.to_datetime(df["registration"])

    # For each user session, we compute the duration in seconds
    df["session_length"] = df.groupby(["userId", "sessionId"])["ts"].transform(
        lambda x: (x.max() - x.min()).total_seconds()
    )

    # Song play indicator:
    df["song_played"] = (df["page"] == "NextSong").astype(int)

    return df

In [3]:
# Aggregating event-level data into user lvl features:


def aggregate_features(data, observation_end):

    observation_end = pd.Timestamp(observation_end)

    # Aggregate features at user level:
    user_df = (
        data.groupby("userId")
        .agg(
            {
                "gender": "first",
                "registration": "first",
                "level": lambda x: x.mode().iloc[0] if not x.mode().empty else 0,
                "sessionId": "nunique",
                "itemInSession": "max",
                "ts": ["min", "max"],
                "session_length": "mean",
                "song_played": "sum",
                "artist": pd.Series.nunique,
                "length": "sum",
            }
        )
        .reset_index()
    )

    # A more intuitive column order and names:
    user_df.columns = [
        "userId",
        "gender",
        "registration",
        "level",
        "num_sessions",
        "max_item_in_session",
        "ts_min",
        "ts_max",
        "avg_session_length_seconds",
        "num_songs_played",
        "unique_artists",
        "total_length",
    ]

    # Creating time based engagement metrics:

    # Days from the first activity to the end of the obs period
    user_df["days_active"] = (observation_end - user_df["ts_min"]).dt.days

    # Total membership duration
    user_df["membership_length"] = (observation_end - user_df["registration"]).dt.days

    # Engagement rates:
    user_df["days_since_last_activity"] = (observation_end - user_df["ts_max"]).dt.days
    user_df["songs_per_day"] = user_df["num_songs_played"] / (
        user_df["days_active"] + 1
    )
    user_df["sessions_per_day"] = user_df["num_sessions"] / (user_df["days_active"] + 1)

    # Fill missing values with 0s
    user_df = user_df.fillna(0)

    # Set userId as index
    user_df.set_index("userId", inplace=True)

    return user_df

Since in this competitions task we are asked to focus on the churn in a specific time window, we create a function which identifies who churned within a specified period.

In [4]:
def get_churned_users(df, start_date, end_date):

    # Convert the given dates to timestamps
    start = pd.Timestamp(start_date)
    end = pd.Timestamp(end_date)

    # Identify users who cancelled their subscirptions in the given period
    cancellations = df[df["page"] == "Cancellation Confirmation"]
    churned = cancellations[
        (cancellations["ts"] > start) & (cancellations["ts"] <= end)
    ]["userId"].unique()

    # Return as a set
    return set(churned)

Now, we are ready to load the train and test data and apply these modifications directly on our datasets.

In [5]:
# Load training data
df_train = import_and_transform("Data/train.parquet")

# Prepare test data
df_test = import_and_transform("Data/test.parquet")

We want to capture more temporal patterns. We want to "teach" the model to recognize churn patterns across different time periods. How do we do that?

If we look at only one point in the time, we do not have enough examples to train the model. So instead of taking just one "snapshot", we take multiple snapshots at different times and we consider each one of them as individual prediction problems. 

Like this, we increase our training data.

In [6]:
# Createing observation dates every 5 days
# Create multiple training samples with sliding window
training_dates = pd.date_range("2018-10-15", "2018-11-05", freq="5D")

X_train_list = []
y_train_list = []

# For each observ date, we create a separate training sample:
for obs_date in training_dates:

    # Filtering data up to the observation date
    df_obs = df_train[df_train["ts"] <= obs_date]
    features = aggregate_features(df_obs, obs_date)

    # Creating a 10 day window after the obervation date
    # And we identify who churned in that period
    window_end = obs_date + pd.Timedelta(days=10)
    churned_users = get_churned_users(df_train, obs_date, window_end)

    # 1 if they churned in the next 10 days, 0 otherwise
    labels = pd.Series(
        features.index.isin(churned_users).astype(int),
        index=features.index,
        name="churned",
    )

    X_train_list.append(features)
    y_train_list.append(labels)

    print(
        f"Date of the observation: {obs_date.date()}, with {len(features)} users, and a {labels.mean():.2%} churn rate"
    )

# We combine all observation windows:
X_train_combined = pd.concat(X_train_list)
y_train_combined = pd.concat(y_train_list)

# Drop non-numeric columns
feature_cols = X_train_combined.select_dtypes(include=[np.number]).columns
feature_cols = [
    c
    for c in feature_cols
    if c not in ["registration", "ts_min", "ts_max", "total_length"]
]

X_train_final = X_train_combined[feature_cols]

test_features = aggregate_features(df_test, "2018-11-20")
X_test = test_features[feature_cols]

Date of the observation: 2018-10-15, with 16271 users, and a 5.08% churn rate
Date of the observation: 2018-10-20, with 17347 users, and a 4.48% churn rate
Date of the observation: 2018-10-25, with 17888 users, and a 4.49% churn rate
Date of the observation: 2018-10-30, with 18271 users, and a 4.46% churn rate
Date of the observation: 2018-11-04, with 18592 users, and a 3.78% churn rate


We are trying to predict 0/1 Yes/No churn. The very first intuitive step to do is to apply Logistic Regression and then optimize it.

Hence, we start by applying a vanilla Logistic Regression model.

In [7]:
log_reg = LogisticRegression(class_weight="balanced")

log_reg.fit(X_train_final, y_train_combined)

from utils import evaluate_model

evaluate_model(log_reg, X_test, 0.5, file_out="log_reg_1.csv")

Base predicted churn: 31.96%
Predicted churn at 0.5 threshold: 31.96%
Submission saved to log_reg_1.csv


c:\Users\andre\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In Logistic Regression, we have hyperprameters that we need to choose *before* the training so we control how the model learns. Our hyperparameters are:

- **C**: how much we penalize complexity (too high is overfitting, too low is underfitting)
- **penalty**: which type of regularization we use (l1 or l2)
- **solver**: which optimization algorithm we use

Since we do not know which combination of these 3 is the best, we need to try all of them to decide which one we'll use in the end.

For solver, we use "Liblinear", and "Saga". A solver is an optimization algorithm that finds the best model param during training. **Saga** is fast, uses low memory and is built for big data. Moreover, it uses Stochastic Average Gradient descent, meaning it updates the model parameters using small bacthes of data at time, it does not load the whole dataset in memory from the beginning, and it converges fast on big data. **Liblinear** is more reliable and is a classic choice.

**ROC-AUC** (area under the curve) measures how well the model separates the 2 classes, despite the imbalance. So in our case, it is used to answer:

"What is the probability the model will rank the churner higher if we pick one churner and one non-churner at random?"

Possible scores:

- $<0.5$ means wrose than random
- $~0.5$ means random guessing
- $1.0$ means perfect

In [8]:
# We define the param grid
param_grid = {
    "C": [0.01, 1, 5, 50], # Regularization strength
    "penalty": ["l1", "l2"], # l1 = Lasso, l2 = Ridge
    "solver": ["liblinear", "saga"], # Optim algorithm
    "class_weight": ["balanced"], # Handle class imbalance
    "max_iter": [1000]
}

# We use cross-validation, because our dataset is imbalanced
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# We perform grid search over all the combinations
# We optimize for ROC-AUC
grid_search = GridSearchCV(
    estimator=LogisticRegression(random_state=42),
    param_grid=param_grid,
    cv=cv,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)

# We train on all the combinations of parameters
grid_search.fit(X_train_final, y_train_combined)

print()
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV ROC-AUC: {grid_search.best_score_:.4f}")

# Let's save the best model now:
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


KeyboardInterrupt: 

We found the best parameters and we will use them to evaluate on the test set.

We got the best ROC-AUC score of 70%, which means that the model learned some real churn patters. However, 30% will miss some churners. The ideal ROC-AUC score for churn prediction should be around 85%.

Now that we have the best model, let's evaluate it on our test set:

In [ ]:
evaluate_model(best_model, X_test, 0.5, file_out="log_reg_2.csv")

Base predicted churn: 61.78%
Predicted churn at 0.5 threshold: 61.78%
Submission saved to log_reg_2.csv


In [9]:
from utils import aggregate_features_improved
# Createing observation dates every 5 days
# Create multiple training samples with sliding window
training_dates = pd.date_range("2018-10-15", "2018-11-05", freq="5D")

X_train_list = []
y_train_list = []

# For each observ date, we create a separate training sample:
for obs_date in training_dates:

    # Filtering data up to the observation date
    df_obs = df_train[df_train["ts"] <= obs_date]
    features = aggregate_features_improved(df_obs, obs_date)

    # Creating a 10 day window after the obervation date
    # And we identify who churned in that period
    window_end = obs_date + pd.Timedelta(days=10)
    churned_users = get_churned_users(df_train, obs_date, window_end)

    # 1 if they churned in the next 10 days, 0 otherwise
    labels = pd.Series(
        features.index.isin(churned_users).astype(int),
        index=features.index,
        name="churned",
    )

    X_train_list.append(features)
    y_train_list.append(labels)

    print(
        f"Date of the observation: {obs_date.date()}, with {len(features)} users, and a {labels.mean():.2%} churn rate"
    )

# We combine all observation windows:
X_train_combined = pd.concat(X_train_list)
y_train_combined = pd.concat(y_train_list)

# Drop non-numeric columns
feature_cols = X_train_combined.select_dtypes(include=[np.number]).columns
feature_cols = [
    c
    for c in feature_cols
    if c not in ["registration", "ts_min", "ts_max", "total_length"]
]

X_train_final = X_train_combined[feature_cols]

test_features = aggregate_features_improved(df_test, "2018-11-20")
X_test = test_features[feature_cols]

Date of the observation: 2018-10-15, with 16271 users, and a 5.08% churn rate
Date of the observation: 2018-10-20, with 17347 users, and a 4.48% churn rate
Date of the observation: 2018-10-25, with 17888 users, and a 4.49% churn rate
Date of the observation: 2018-10-30, with 18271 users, and a 4.46% churn rate
Date of the observation: 2018-11-04, with 18592 users, and a 3.78% churn rate


In [10]:
X_train_final.head()

,gender,level_first,level_current,num_sessions,avg_session_length,num_songs_played,unique_artists,unique_songs,avg_song_length,days_active,...,downgrade_attempts,upgrade_attempts,total_page_views,has_social_activity,positive_actions,satisfaction_ratio,engagement_rate,problem_signals,ads_per_song,actions_per_session
userId,,,,,,,,,,,,,,,,,,,,,
1000025,1,0,1,11,56774.847498,1030,795,935,249.797113,12,...,14.0,2.0,1259,1,121.0,0.864198,0.131911,0.0,0.003880,104.916667
1000035,0,0,0,4,12457.375000,66,66,66,253.653276,9,...,0.0,1.0,88,0,0.0,0.000000,0.000000,0.0,0.029851,17.600000
1000083,1,0,1,11,22024.669463,501,427,478,244.723096,13,...,2.0,4.0,596,1,35.0,0.875000,0.077689,0.0,0.015936,49.666667
1000103,0,0,1,1,9744.000000,39,39,39,250.488380,10,...,0.0,2.0,51,0,0.0,0.000000,0.000000,0.0,0.075000,25.500000
1000164,0,0,0,7,11326.024096,127,119,125,244.640807,13,...,0.0,0.0,166,1,13.0,0.000000,0.000000,0.0,0.140625,20.750000


In [12]:
log_reg = LogisticRegression(class_weight="balanced")

log_reg.fit(X_train_final, y_train_combined)
evaluate_model(log_reg, X_test, 0.5, file_out='new_log_reg.csv')

Base predicted churn: 53.06%
Predicted churn at 0.5 threshold: 53.06%
Submission saved to new_log_reg.csv


c:\Users\andre\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
